Rodzaje ruchu:\
Swap - losujemy dwa miasta i zamieniamy je ze sobą.\
Insercja - losujemy miasto i punkt w którym je umieszczamy.\
Zamiana kolejności - losujemy dwa punkty i między nimi zamieniamy kolejność.\
\
Sąsiedztwo to dwa przypadki które różnią się między sobą jednym ruchem\
\
Klasyczny algorytm wspinaczki z multistartem: po każdym ruchu liczymy drogę i sprawdzamy czy jest mniejsza czy nie, wybieramy tylko lepsze opcje. Multistart polega na wielokrotnym odpaleniu algorytmu z losowaniem punktu startowego.\
\
Algorytm symulowanego wyżarzania: zmodyfikowany algorytm wspinaczki, jeśli znalezione rozwiązanie jest gorsze to nadal je rozważamy, na podstawie tego jak bardzo jest gorsze oraz od temperatury.\
Temperatura jest wysoka na początku działania algorytmu i spada razem z czasem wykonywania algorytmu, im niższa temperatura tym mniejsze prawdopodobieństwo wybrania gorszego rozwiązania, bo algorytm już trochę działa i zakładamy że jest w lepszym miejscu niż na początku.\
Pozwala on nam wyjść z niektórych ekstremów lokalnych dzięki możliwości przyjęcia gorszego rozwiązania.\
\
Tabu search:\
Sąsiedztwo wygląda tutaj inaczej: przy swapie losujemy jedną liczbę i zamieniamy wszystkie kombinacje z tą liczbą, ze wszystkich kombinacji wybieramy najlepszą.\
Następnie taka kombinacja trafia na listę tabu, czyli takiej samej zamiany nie można wykonać przez określoną kolejną liczbę ruchów.\
\
Funkcja aspiracji to funkcja, która pod pewnymi warunkami pozwala wykonać ruch z listy tabu ale nie trzeba tego implementować.\
\
Odpowiednia długość listy tabu pozwala wyjść z niektórych ekstremów lokalnych, jeśli będzie za krótka to możemy nie wyjść z ekstremum lokalnego, jeśli będzie za długa to możemy wyjść z ekstremum niepotrzebnie.


In [25]:
import numpy as np
import pandas as pd
import math
import random

In [26]:
#wspinaczka z multistartem
distance = np.array([[0,2,4,1,5],
                    [2,0,3,8,7],
                    [4,3,0,5,1],
                    [1,8,5,0,3],
                    [5,7,1,3,0]])

In [27]:
def Odleglosc(distance, trasa):
    suma = 0
    n = len(trasa)
    
    for i, current_city in enumerate(trasa):
        next_city = trasa[(i + 1) % n]  # Use modulo to handle the last element
        suma += distance[current_city][next_city]

    return suma

In [28]:
#swap
def Swap(trasa):
    trasa_temp = trasa.copy()
    #print(trasa)
    swap = np.random.choice(trasa_temp,2,replace=False)
    temp = trasa_temp[swap[0]]
    trasa_temp[swap[0]] = trasa_temp[swap[1]]
    trasa_temp[swap[1]] = temp
    #print(swap)
    #print(trasa)
    return trasa_temp

#insercja
def Insercja(trasa):
    trasa_temp = trasa.copy()
    insercja = np.random.choice(trasa_temp,2,replace=False)
    #print(trasa_temp)
    #print(insercja)
    temp = trasa_temp[insercja[0]]
    trasa_temp = np.delete(trasa_temp,insercja[0])
    trasa_temp = np.insert(trasa_temp,insercja[1],temp)
    #print(trasa_temp)
    return trasa_temp

def Zamiana(trasa):
    trasa_temp = trasa.copy()
    zamiana = np.random.choice(trasa_temp,2,replace=False)
    zamiana.sort()
    #print(trasa_temp)
    #print(zamiana)
    for i in range(math.ceil((zamiana[1]-zamiana[0])/2)):
        temp = trasa_temp[zamiana[1]]
        trasa_temp[zamiana[1]] = trasa_temp[zamiana[0]]
        trasa_temp[zamiana[0]] = temp
        zamiana[1] -= 1
        zamiana[0] += 1
    #print(trasa_temp)
    return trasa_temp

In [29]:
#Zamiana(trasa)

In [30]:
#trasa = np.arange(len(distance))
#np.random.shuffle(trasa)
#Insercja(trasa)

In [31]:
#temp = trasa[0] #temp = 4
#trasa = np.delete(trasa,0)
#trasa = np.insert(trasa,4,3)

In [32]:
#ruch polegający na swapie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_swap(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Swap(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new
    return trasa, odl, trasa_new, odl_new

#ruch polegający na insercji, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_insercja(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Insercja(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new
    return trasa, odl

#ruch polegający na zamianie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_zamiana(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Zamiana(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new  
    return trasa, odl

In [33]:
distance = pd.read_excel("Dane_TSP_48.xlsx")
distance = distance.to_numpy()
distance = np.delete(distance,0,1)
distance

array([[   0, 1495,  381, ...,  488,  753, 1184],
       [1495,    0, 1135, ..., 1727, 1388,  661],
       [ 381, 1135,    0, ...,  640,  591,  810],
       ...,
       [ 488, 1727,  640, ...,    0,  520, 1242],
       [ 753, 1388,  591, ...,  520,    0,  801],
       [1184,  661,  810, ..., 1242,  801,    0]], dtype=int64)

In [34]:
def IHC(distance, ruch, n_iter_ruch, n_iter):  # ruch = {Swap, Insercja, Zamiana}
    min_iter = []
    for i in range(n_iter):
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                if odl_new < odl:
                    odl = odl_new
                    trasa = trasa_new
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = IHC(distance,Swap,1000,100)
wynik.sort(key=lambda x: x[2])
wynik[0:10]



[(55,
  array([23,  9,  3, 25, 34, 44, 31, 21, 15, 40, 28,  1, 41,  4, 47, 10, 11,
         27, 26, 18, 42, 16, 36, 45, 14, 39, 13, 12, 24, 38, 20, 46, 32, 19,
         30, 43, 17,  6,  5, 29, 35, 37,  8,  0,  7,  2, 22, 33]),
  16516),
 (9,
  array([42, 20, 31, 38, 24, 13, 23, 47, 41,  9, 34, 44,  3, 25,  4, 22, 12,
         10, 21,  2, 15,  7, 37, 30, 14, 32, 39,  0, 40, 28,  1, 33, 46, 19,
         11,  8, 45, 17, 29, 26, 18, 36,  5, 27,  6, 43, 35, 16]),
  16844),
 (23,
  array([35, 36, 16, 42, 29,  6, 17, 30, 43, 45, 32, 31, 12, 38, 34, 44,  3,
          1, 28, 33,  4, 23,  9, 25, 41, 47, 24, 13, 22, 20, 46, 19, 11, 18,
         26,  5, 27, 37,  8,  7, 15, 40, 21,  2, 10,  0, 39, 14]),
  16898),
 (87,
  array([43, 30, 40, 28,  1,  4, 38, 31, 44, 34,  3, 25, 10, 39, 11, 14, 45,
         37,  8, 19, 46, 20, 12, 24,  9, 23, 41, 47, 33, 21,  7, 15,  0,  2,
         13, 22, 32, 35,  5, 36, 18, 16, 26, 42, 29, 27, 17,  6]),
  16917),
 (98,
  array([43, 30, 37,  6,  7,  0,  1, 28,  4, 41

In [35]:
def Geometric(T,alpha):
    return alpha*T

def Slow(T,alpha):
    return T/(1+alpha*T)


In [36]:
def SA(distance, ruch, n_iter_ruch, n_iter, T_start, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = SA(distance,Swap,1000,100,100,Slow,0.1)
wynik.sort(key=lambda x: x[2])
wynik[0:10]

[(34,
  array([43, 30, 17,  6, 35, 32, 46, 24,  4, 13,  2, 15, 21, 40, 33,  1, 28,
         38, 23,  9, 25,  3, 44, 34, 41, 47, 12, 20, 31, 29, 26, 16, 42, 18,
          5, 36, 27, 45, 19, 10, 11, 14, 22, 39,  0,  7,  8, 37]),
  15938),
 (74,
  array([29,  6, 11, 31, 38, 47, 23,  9, 25,  3, 34, 44, 41, 12, 20, 46, 10,
         45, 17, 27,  5, 43, 37,  8, 28,  1,  4, 33,  2, 22, 24, 13, 40, 15,
         21, 30, 19, 32, 14,  7,  0, 39, 35, 18, 36, 26, 16, 42]),
  17190),
 (63,
  array([30, 36,  5, 16, 42, 46, 20, 12, 33, 40, 19, 26, 18,  7,  0, 15,  2,
         22, 21, 39, 35, 27, 29, 45, 32, 14, 11, 31, 23,  9, 41, 44, 34,  3,
         25,  1, 28, 47,  4, 38, 24, 13, 10, 17,  6, 43,  8, 37]),
  17490),
 (37,
  array([21, 33, 28, 23, 44,  9, 41,  4, 11, 39, 45, 14,  8,  7, 13, 24, 22,
         10, 32, 29, 26, 18, 36,  5, 27, 35, 17, 43, 37,  0,  2, 15, 40,  1,
          3, 25, 34, 47, 38, 31, 12, 46, 20, 19, 42, 16,  6, 30]),
  17531),
 (12,
  array([23, 41, 47,  4, 24, 14, 37, 30, 43, 3

In [37]:
# wersja SA gdzie warunkiem stopu jest temperatura a nie liczba iteracji 
# (chyba lepiej tak bo upewniamy sie ze temperatura jest niska a podajac liczbe iteracji mozemy skonczyc na wysokiej temperaturze)
# mozna tez dodac warunek stopu ze po np 100 iteracjach bez poprawy rozwiazania to koniec

def SA(distance, ruch, n_iter, T_start, T_stop, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        while(T > T_stop):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = SA(distance,Swap,100,100,0.01,Slow,0.1)
wynik.sort(key=lambda x: x[2])
wynik[0:10]

[(11,
  array([40,  1, 28, 41,  4, 47, 31, 23, 44,  9, 34,  3, 25, 33,  2, 13, 22,
         12, 20, 38, 24, 10, 11, 19, 46, 14, 37,  8,  7,  0, 39, 32, 35,  6,
          5, 18, 16, 36, 17, 43, 26, 42, 29, 27, 45, 30, 21, 15]),
  15490),
 (93,
  array([ 0,  2, 13, 38, 47, 31, 20, 19, 35, 27,  6, 30, 37,  8, 45, 29, 26,
         42, 16, 18, 36,  5, 17, 32, 43, 14, 39, 10, 22,  7, 15, 40, 41, 44,
         34,  1,  3, 25,  9, 23,  4, 28, 33, 24, 12, 46, 11, 21]),
  16549),
 (47,
  array([ 3,  1, 40,  2, 22, 28,  4, 13, 24, 20, 31, 44, 34, 41, 38, 12, 29,
         36, 35, 27,  5, 18, 26, 16, 42, 45,  8,  7,  0, 21, 15, 10, 46, 19,
         11, 14, 32, 17,  6, 43, 30, 37, 39, 33, 47, 23,  9, 25]),
  16665),
 (43,
  array([42, 18, 36,  5, 22,  2, 12, 24, 14, 45, 30, 37, 39, 11, 32, 35,  6,
         43, 46, 10, 13, 33, 40, 15, 21,  0,  7,  8, 28, 41, 25,  3,  1,  4,
         47, 38, 31, 23, 44, 34,  9, 20, 19, 16, 26, 27, 17, 29]),
  16761),
 (19,
  array([18, 26, 16, 42, 29,  6, 43, 45, 14, 3

In [38]:
# funkcje pomocnicze do sprawdzenia ile iteracji odpowiada parametrowi T_stop
def T_stopToIter(T_start, T_stop, cooling, alpha):
    T = T_start
    i = 0
    while(T > T_stop):
        T = cooling(T,alpha)
        i += 1
    return i
def IterToT_stop(T_start, n_iter, cooling, alpha):
    T = T_start
    for i in range(n_iter):
        T = cooling(T,alpha)
    return T
print(T_stopToIter(100,0.01,Slow,0.1))
IterToT_stop(100,100,Slow,0.1)

1000


0.09990009990009993

In [39]:
# #algorytm ze swapem
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 5

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_swap(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))


In [40]:
# #algorytm z insercją
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 5

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_insercja(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))

In [41]:
# #algorytm z zamianą
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 1000

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_zamiana(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))

In [137]:
#NN
min_odl = []
for n in range(100):
    potencjalne = np.arange(len(distance))
    trasa = np.zeros(len(distance),dtype=int)
    trasa[0] = np.random.choice(potencjalne,1)
    potencjalne = np.delete(potencjalne,int(trasa[0]))

    for cur in range(1,len(trasa)):
        odleglosci = []
        for i in range(len(potencjalne)):
            trasa[cur] = potencjalne[i]
            odleglosci.append((potencjalne[i],Odleglosc(distance,trasa[0:cur+1])))
        odleglosci.sort(key=lambda x: x[1])
        trasa[cur] = odleglosci[0][0]
        potencjalne = np.delete(potencjalne,potencjalne==trasa[cur])
    min_odl.append((Odleglosc(distance,trasa),trasa))
    
min_odl.sort(key=lambda x:x[0])
min_odl[:10]

[(13251,
  array([ 8,  0,  7, 37, 30, 45, 14, 39, 11, 32, 43, 17,  6, 35, 27,  5, 29,
         36, 18, 26, 42, 16, 19, 10, 22,  2, 21, 15, 40, 33, 13, 12, 46, 20,
         24, 38, 31, 47,  4, 28,  1, 41, 23,  9, 25,  3, 34, 44])),
 (13251,
  array([ 8,  0,  7, 37, 30, 45, 14, 39, 11, 32, 43, 17,  6, 35, 27,  5, 29,
         36, 18, 26, 42, 16, 19, 10, 22,  2, 21, 15, 40, 33, 13, 12, 46, 20,
         24, 38, 31, 47,  4, 28,  1, 41, 23,  9, 25,  3, 34, 44])),
 (13251,
  array([ 8,  0,  7, 37, 30, 45, 14, 39, 11, 32, 43, 17,  6, 35, 27,  5, 29,
         36, 18, 26, 42, 16, 19, 10, 22,  2, 21, 15, 40, 33, 13, 12, 46, 20,
         24, 38, 31, 47,  4, 28,  1, 41, 23,  9, 25,  3, 34, 44])),
 (13414,
  array([44, 34,  9, 23, 41, 25,  3,  1, 28,  4, 47, 38, 31, 24, 13, 12, 20,
         46, 10, 22, 33,  2, 21, 15, 40, 39, 11, 19, 32, 14, 45,  8,  0,  7,
         37, 30, 43, 17, 35,  6, 27, 29, 42, 26,  5, 36, 18, 16])),
 (13414,
  array([44, 34,  9, 23, 41, 25,  3,  1, 28,  4, 47, 38, 31, 24, 13